In [ ]:
import numpy as np
import pandas as pd
import spacy
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df=pd.read_csv("/content/drive/MyDrive/training.csv",encoding='latin-1', header=None)

In [ ]:
df.head(5)

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
df.columns = ['target', 'ids', 'date', 'flag', 'user', 'text']

In [ ]:
df = df[['target', 'text']]
df['target'] = df['target'].replace({4: 1})

<ipython-input-6-8c526963b768>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target'] = df['target'].replace({4: 1})


In [ ]:

df_class_0 = df[df['target'] == 0].sample(n=200000, random_state=42)
df_class_1 = df[df['target'] == 1].sample(n=200000, random_state=42)

# Combining the two samples into one DataFrame
df_sampled = pd.concat([df_class_0, df_class_1])

# Optionally, shuffle the combined DataFrame
df_sampled = df_sampled.sample(frac=1, random_state=42).reset_index(drop=True)

df_sampled.target.value_counts()

,count
target,
0,200000
1,200000


In [ ]:
nlp = spacy.load('en_core_web_sm')
def preprocess(text):
  doc=nlp(text)
  tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
  return ' '.join(tokens)


In [ ]:
df_sampled["clean_text"]=df_sampled["text"].apply(preprocess)

In [ ]:
import re

def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove special characters and numbers, keeping only letters and spaces
    text = re.sub(r'[^A-Za-z\s]', '', text)
    # Remove extra spaces
    text = text.strip()
    return text

In [ ]:
df_sampled['clean_text'] = df_sampled['clean_text'].apply(clean_text)

In [ ]:
df_sampled.head(5)

,target,text,clean_text
0,0,would like to settle down and watch the footba...,like settle watch football beer fat bloody chance
1,0,reaalllly hungry. want pho. and/or fried chick...,reaalllly hungry want pho andor fry chicken an...
2,0,@fiftypavements they don't air here so I have ...,fiftypavement air watch YouTube
3,0,is waiting for Kristin's call.... need to go o...,wait Kristin need drive nut day leave long
4,1,"Susan Boyle(Bwaakk), Google Wave, BGT, Diversi...",Susan BoyleBwaakk Google Wave BGT Diversity La...


In [ ]:
#TF-IDF Vectorization:

from sklearn.feature_extraction.text import TfidfVectorizer

# Create TF-IDF features
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(df_sampled['clean_text'])

In [ ]:
#ngrams Victorization
ngram_vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X_ngrams = ngram_vectorizer.fit_transform(df_sampled['clean_text'])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, df_sampled['target'], test_size=0.2, random_state=42)

# Train the model
model = MultinomialNB()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.7432375
              precision    recall  f1-score   support

           0       0.72      0.78      0.75     39866
           1       0.77      0.70      0.73     40134

    accuracy                           0.74     80000
   macro avg       0.74      0.74      0.74     80000
weighted avg       0.74      0.74      0.74     80000



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_ngrams, df_sampled['target'], test_size=0.2, random_state=42)

# Train the model
model = MultinomialNB()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.7567
              precision    recall  f1-score   support

           0       0.74      0.79      0.76     39866
           1       0.78      0.72      0.75     40134

    accuracy                           0.76     80000
   macro avg       0.76      0.76      0.76     80000
weighted avg       0.76      0.76      0.76     80000

